In [1]:
import pandas as pd
import mysql.connector

In [2]:
df = pd.read_csv('superstore_Orders.csv', encoding='ISO-8859-1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
#convert order date and ship date to datetime from format dd/mm/yyyy
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y')

#add a column for the delivery time as an integer of days
df['Delivery Time'] = (df['Ship Date'] - df['Order Date']).dt.days


In [4]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Delivery Time
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,3
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,3
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,4
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,7
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,7


In [5]:
db = mysql.connector.connect(
    host="localhost",
    user="",
    passwd=""
)

In [9]:
#create database to upload the csv
mycursor = db.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS superstore")
mycursor.execute("USE superstore")
#Create table to upload the csv

mycursor.execute("CREATE TABLE IF NOT EXISTS orders ("
                    "Row_ID INT(5) NOT NULL,"
                    "Order_ID VARCHAR(15) NOT NULL,"
                    "Order_Date DATE NOT NULL,"
                    "Ship_Date DATE NOT NULL,"
                    "Ship_Mode VARCHAR(30) NOT NULL,"
                    "Customer_ID VARCHAR(10) NOT NULL,"
                    "Customer_Name VARCHAR(50) NOT NULL,"
                    "Segment VARCHAR(35) NOT NULL,"
                    "Country VARCHAR(100) NOT NULL,"
                    "City VARCHAR(100) NOT NULL,"
                    "State VARCHAR(100) NOT NULL,"
                    "Postal_Code VARCHAR(8) NOT NULL,"
                    "Region VARCHAR(25) NOT NULL,"
                    "Product_ID VARCHAR(25) NOT NULL,"
                    "Category VARCHAR(100) NOT NULL,"
                    "Sub_Category VARCHAR(100) NOT NULL,"
                    "Product_Name VARCHAR(255) NOT NULL,"
                    "Sales FLOAT(11) NOT NULL,"
                    "Quantity INT(11) NOT NULL,"
                    "Discount FLOAT(11) NOT NULL,"
                    "Profit FLOAT(11) NOT NULL,"
                    "Delivery_Time INT(11) NOT NULL,"
                    "PRIMARY KEY (Row_ID)"
                    ")")



# Upload the CSV to the database
for index, row in df.iterrows():
    sql = "INSERT INTO orders (Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode, Customer_ID, Customer_Name, Segment, Country, City, State, Postal_Code, Region, Product_ID, Category, Sub_Category, Product_Name, Sales, Quantity, Discount, Profit, Delivery_Time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s)"

    values = (row['Row ID'], row['Order ID'], row['Order Date'], row['Ship Date'], row['Ship Mode'], row['Customer ID'], row['Customer Name'], row['Segment'], row['Country'], row['City'], row['State'], row['Postal Code'], row['Region'], row['Product ID'], row['Category'], row['Sub-Category'], row['Product Name'], row['Sales'], row['Quantity'], row['Discount'], row['Profit'], row['Delivery Time'])
    
    mycursor.execute(sql, values)


#db.commit()
print("CSV data uploaded successfully to the database.")

CSV data uploaded successfully to the database.


In [10]:
#check if the data was uploaded correctly by counting the rows
mycursor.execute("SELECT COUNT(*) FROM orders")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

#check if the data was uploaded correctly by showing the first 5 rows
mycursor.execute("SELECT * FROM orders LIMIT 5")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)


(9994,)
(1, 'CA-2016-152156', datetime.date(2016, 11, 8), datetime.date(2016, 11, 11), 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', '42420', 'South', 'FUR-BO-10001798', 'Furniture', 'Bookcases', 'Bush Somerset Collection Bookcase', 261.96, 2, 0.0, 41.9136, 3)
(2, 'CA-2016-152156', datetime.date(2016, 11, 8), datetime.date(2016, 11, 11), 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', '42420', 'South', 'FUR-CH-10000454', 'Furniture', 'Chairs', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 731.94, 3, 0.0, 219.582, 3)
(3, 'CA-2016-138688', datetime.date(2016, 6, 12), datetime.date(2016, 6, 16), 'Second Class', 'DV-13045', 'Darrin Van Huff', 'Corporate', 'United States', 'Los Angeles', 'California', '90036', 'West', 'OFF-LA-10000240', 'Office Supplies', 'Labels', 'Self-Adhesive Address Labels for Typewriters by Universal', 14.62, 2, 0.0, 6.8714, 4)
(4, 'US-2015-108966', dat

In [11]:
#commit the changes to the database
db.commit()
db.close()